In [1]:
import pandas as pd
from datetime import timedelta, datetime
import osmnx as ox
import networkx as nx

In [2]:
# Координаты бригад и предварительная загрузка данных
brigades = {
    "A": (55.818119, 37.578991), # Яблочкова
    "B": (55.753185, 37.410267), # Осенний
    "C": (55.793629, 37.708969), # Электрозаводская
    "D": (55.638885, 37.761070), # Паромная
    "D_new": (55.837784, 37.655743) # Дорожный
}

In [3]:
# Загрузка данных инцидентов
incidents = pd.read_excel('2023new.xlsx')

In [4]:
# Обработка данных инцидентов
incidents['Широта'] = pd.to_numeric(incidents['Широта'], errors='coerce')
incidents['Долгота'] = pd.to_numeric(incidents['Долгота'], errors='coerce')
incidents.dropna(subset=['Широта', 'Долгота'], inplace=True)

In [5]:
# Инициализация графа дорог для Москвы
G = ox.graph_from_place('Moscow, Russia', network_type='drive')
G = ox.project_graph(G)
G = nx.convert_node_labels_to_integers(G)

In [6]:
# Функции для расчетов
def get_shortest_path_length(G, start_point, end_point):
    start_node = ox.distance.nearest_nodes(G, start_point[1], start_point[0])
    end_node = ox.distance.nearest_nodes(G, end_point[1], end_point[0])
    length = nx.shortest_path_length(G, start_node, end_node, weight='length')
    return length / 1000

In [7]:

def calculate_travel_time(distance, speed=40):
    return distance / speed

brigade_availability = {brigade: pd.Timestamp.min for brigade in brigades.keys()}

In [8]:

def get_brigade_coords(brigade_name, incident_date):
    if brigade_name == 'D' and incident_date >= datetime(2023, 12, 15):
        return brigades['D_new']
    else:
        return brigades[brigade_name]

In [9]:

def adjust_start_time_for_shift_change(incident_time, brigade_name):
    shift_start_hour = 9 if brigade_name in ['A', 'C'] else 8
    shift_change_time = datetime(incident_time.year, incident_time.month, incident_time.day, shift_start_hour)
    if incident_time < shift_change_time:
        return max(incident_time, shift_change_time + timedelta(minutes=30))
    else:
        return incident_time + timedelta(minutes=30)

In [10]:

def assign_incidents_to_brigades(incidents, brigades, G, brigade_availability):
    assignments = []
    for _, incident in incidents.iterrows():
        incident_coord = (incident['Широта'], incident['Долгота'])
        incident_date = pd.to_datetime(incident['Т события'])
        nearest_brigade, min_distance = None, float('inf')
        best_end_time = None
        for brigade_name, _ in brigades.items():
            brigade_coord = get_brigade_coords(brigade_name, incident_date)
            corrected_start_time = adjust_start_time_for_shift_change(incident_date, brigade_name)
            path_length = get_shortest_path_length(G, brigade_coord, incident_coord)
            travel_time = timedelta(hours=calculate_travel_time(path_length))
            start_work_time = max(corrected_start_time, brigade_availability[brigade_name]) + travel_time
            end_work_time = start_work_time + timedelta(minutes=30)
            if best_end_time is None or end_work_time < best_end_time:
                nearest_brigade = brigade_name
                best_end_time = end_work_time
        if nearest_brigade:
            brigade_availability[nearest_brigade] = best_end_time
            # В выходных данных бригада D_new обозначается как D
            display_brigade_name = "D" if nearest_brigade == "D_new" else nearest_brigade
            assignments.append((incident['№ МИ'], display_brigade_name, start_work_time, best_end_time))
    return assignments

In [11]:

def distribute_daily_incidents(incidents, brigades, start_date, end_date, G):
    daily_assignments = {}
    for single_date in pd.date_range(start=start_date, end=end_date):
        daily_incidents = incidents[incidents['Т события'].dt.date == single_date.date()]
        if not daily_incidents.empty:
            brigade_availability = {brigade: pd.Timestamp.min for brigade in brigades.keys()}  # Сброс доступности бригад каждый день
            daily_assignments[single_date.date()] = assign_incidents_to_brigades(daily_incidents, brigades, G, brigade_availability)
    return daily_assignments

In [12]:

def save_assignments(daily_distributions, filename='daily_assignments.csv'):
    all_assignments = []
    for day, assignments in daily_distributions.items():
        for incident_id, brigade, start_time, end_time in assignments:
            incident_record = incidents[incidents['№ МИ'] == incident_id].iloc[0]
            all_assignments.append({
                'Date': day.strftime('%Y-%m-%d'),
                'Incident ID': incident_id,
                'Assigned Brigade': brigade,
                'Incident Type': incident_record['Техописание'],
                'Time Assigned': incident_record['Т события'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(incident_record['Т события']) else 'N/A',
                'Estimated End Time': end_time.strftime('%Y-%m-%d %H:%M:%S'),
                'Deadline': incident_record['Крайний срок'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(incident_record['Крайний срок']) else 'N/A'
            })
    df = pd.DataFrame(all_assignments)
    df.to_csv(filename, index=False)
    print(f'Saved assignments to {filename}')

In [13]:

# Пример использования
# Загрузка данных инцидентов
# incidents = pd.read_excel('2023.xlsx')
# Подготовка данных инцидентов
incidents['Т события'] = pd.to_datetime(incidents['Т события'])
start_date = incidents['Т события'].min()
end_date = incidents['Т события'].max()
# Список фраз, с которых не должны начинаться записи в столбце "Техописание"
exclude_starts = [
    "BatteryOffline",
    "Требуется замена акб",
    "BatteryPowerOFF",
    "Отключены батареи",
    "Подозрение на кражу АКБ",
    "BatteryLoopOffline"
]

# Фильтрация строк, которые начинаются с указанных фраз
mask = ~incidents['Техописание'].str.startswith(tuple(exclude_starts), na=False)
incidents = incidents[mask]

# Распределение инцидентов и сохранение результатов
daily_distributions = distribute_daily_incidents(incidents, brigades, start_date, end_date, G)
save_assignments(daily_distributions, 'version_3.2.csv')

KeyboardInterrupt: 

In [ ]:
# Загрузка данных инцидентов
incidents = pd.read_excel('2023.xlsx')  # Убедитесь, что путь к файлу верный
# Подготовка данных инцидентов
incidents['Т события'] = pd.to_datetime(incidents['Т события'])
start_date = incidents['Т события'].min()
end_date = incidents['Т события'].max()
# Список фраз, с которых не должны начинаться записи в столбце "Техописание"
exclude_starts = [
    "BatteryOffline",
    "Требуется замена акб",
    "BatteryPowerOFF",
    "Отключены батареи",
    "Подозрение на кражу АКБ",
    "BatteryLoopOffline"
]

# Фильтрация строк, которые начинаются с указанных фраз
mask = ~incidents['Техописание'].str.startswith(tuple(exclude_starts), na=False)
incidents = incidents[mask]